In [399]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import robust_scale
from sklearn.cluster import HDBSCAN
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cdist

## Importazione dati di lavoro
Questa sezione andrà riscritta quando faremo i collegamenti client-server e da/verso il database

### Importiamo i dati
Qui sarà la parte dove importiamo i dati dal database

In [400]:
df1 = pd.read_csv("/home/edi/Scrivania/uni/Mag/DDSE/QPID/qpid/backend/dataset.csv")
df = df1[0:8637]
df = df.drop(columns=["gender", "age_interval", "gender_text", "gender_partner_text"])
df.describe()

/tmp/ipykernel_15149/3748830538.py:1: DtypeWarning: Columns (34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("/home/edi/Scrivania/uni/Mag/DDSE/QPID/qpid/backend/dataset.csv")


,ID,age_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,age,attractive,sincere,...,tv,theater,movies,concerts,music,shopping,yoga,arts,longitude,latitude
count,8637.000000,8637.000000,8637.000000,8637.000000,8637.000000,8637.000000,8637.000000,8637.000000,8637.000000,8637.00000,...,8637.000000,8637.000000,8637.000000,8637.000000,8637.000000,8637.000000,8637.000000,8637.000000,8637.000000,8637.000000
mean,4318.000000,26.375478,1.132685,0.858747,1.012620,0.861410,0.442283,26.361931,3.644784,4.06530,...,2.614334,3.378604,3.923006,6.829223,3.649415,2.791015,2.094361,3.416927,26.799949,68.535548
std,2493.431471,3.534933,0.711998,0.499846,0.461037,0.463477,0.507981,3.549403,0.744767,0.65253,...,1.360221,1.134635,0.806542,2.143398,0.912348,1.357982,1.487951,1.068825,438.533485,1052.191116
min,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000,1.000000,1.00000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,7.388362,37.209845
25%,2159.000000,24.000000,1.000000,1.000000,1.000000,1.000000,0.000000,24.000000,3.000000,4.00000,...,2.000000,2.000000,4.000000,5.000000,3.000000,2.000000,1.000000,3.000000,10.240712,41.042558
50%,4318.000000,26.000000,1.000000,1.000000,1.000000,1.000000,0.000000,26.000000,4.000000,4.00000,...,3.000000,4.000000,4.000000,7.000000,4.000000,3.000000,2.000000,4.000000,11.719424,44.122764
75%,6477.000000,28.000000,1.000000,1.000000,1.000000,1.000000,1.000000,28.000000,4.000000,4.00000,...,4.000000,4.000000,4.000000,8.000000,4.000000,4.000000,3.000000,4.000000,13.972858,45.337627
max,8636.000000,55.000000,5.000000,3.000000,2.000000,2.000000,3.000000,55.000000,5.000000,5.00000,...,5.000000,5.000000,5.000000,10.000000,5.000000,5.000000,5.000000,5.000000,16897.000000,45592.000000


### Creazione del punto di ricerca
Per il momento prendiamo un dato a caso dal dataset come nostro candidato

In [401]:
search = df.sample()
print(search)

        ID  age_o  attractive_important  sincere_important  \
7543  7543   24.0                   1.0                1.0   

      intellicence_important  funny_important  ambtition_important   age  \
7543                     1.0              1.0                  0.0  29.0   

      attractive  sincere  ...   tv  theater  movies  concerts  music  \
7543         2.0      5.0  ...  4.0      4.0     4.0       8.0    4.0   

      shopping  yoga  arts  longitude   latitude  
7543       4.0   2.0   2.0   8.734348  45.157944  

[1 rows x 33 columns]


Costruiamo un punto ad-hoc e salviamo i pesi

In [402]:
search["age"] = search["age_o"]
weights = search.iloc[:, 2:7].to_numpy()[0]
weights[weights > 4] = 4

In [403]:
search = search.drop(columns=[
    "attractive_important", "sincere_important", "intellicence_important",
    "funny_important", "ambtition_important", "age_o"
    ])

In [404]:
search.iloc[:, 2:7] = 0
print(search)
print(weights)

        ID   age  attractive  sincere  intelligence  funny  ambition  sports  \
7543  7543  24.0           0        0             0      0         0     2.0   

      tvsports  exercise  ...   tv  theater  movies  concerts  music  \
7543       2.0       2.0  ...  4.0      4.0     4.0       8.0    4.0   

      shopping  yoga  arts  longitude   latitude  
7543       4.0   2.0   2.0   8.734348  45.157944  

[1 rows x 27 columns]
[1. 1. 1. 1. 0.]


/tmp/ipykernel_15149/1051568922.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  search.iloc[:, 2:7] = 0


Prepariamo il dataset alla ricerca.

Salviamo gli ID utenti per poter recuperare tutte le loro informazioni alla fine
Prepariamo il dataset alla ricerca. 

In [405]:
df = df.drop(index=search.iloc[0, 0])
IDs = df["ID"].to_list()
IDs.append(search.iloc[0,0])
IDs = np.array(IDs)
df = df.drop(columns=[
    "attractive_important", "sincere_important", "intellicence_important",
    "funny_important", "ambtition_important", "age_o", "ID"
])
print(df.columns)
search = search.drop(columns=["ID"])

Index(['age', 'attractive', 'sincere', 'intelligence', 'funny', 'ambition',
       'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking',
       'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts',
       'music', 'shopping', 'yoga', 'arts', 'longitude', 'latitude'],
      dtype='object')


Convertiamo tutto in numpy array per un'elaborazione numerica più facile, il punto di ricerca sarà l'ultimo punto

In [406]:
search = search.to_numpy()[0]
X = df.to_numpy()
X = np.vstack((X,search))
print("X:",X, X.shape)

X: [[25.        3.        4.       ...  5.       12.576222 37.878934]
 [24.        4.        4.       ...  2.        8.741729 44.011987]
 [24.        2.        4.       ...  2.       11.532011 43.4853  ]
 ...
 [23.        2.        4.       ...  4.       10.530364 43.546925]
 [25.        4.        3.       ...  4.       13.564737 45.900195]
 [24.        0.        0.       ...  2.        8.734348 45.157944]] (8637, 26)


## Clustering

In [407]:
clustering = HDBSCAN(min_cluster_size=5, n_jobs=-1, store_centers="medoid", allow_single_cluster=True)
clustering.fit(X[:, np.r_[0, 6:26]])

# ri-assegnamo i punti rumorosi
medoids = clustering.medoids_
print("Medoids shape:", medoids.shape)
noisy = X[clustering.labels_ == -1]
noisy = noisy[:,np.r_[0,6:26]]
print("Noisy points shape:",noisy.shape)
distmat = cdist(noisy, medoids)
idxs = np.argmin(distmat, axis=1)
clustering.labels_[clustering.labels_ == -1] = idxs

# filtriamo
print("X before:",X.shape)
X = X[clustering.labels_ == clustering.labels_[-1]]
IDs = IDs[clustering.labels_ == clustering.labels_[-1]]
print("X after:",X.shape)
print(X)

Medoids shape: (641, 21)
Noisy points shape: (1174, 21)
X before: (8637, 26)
X after: (17, 26)
[[23.        4.        4.        4.        4.        4.        2.
   3.        4.        4.        4.        4.        1.        0.
   4.        4.        2.        4.        4.        7.        4.
   3.        2.        2.        8.320688 44.436916]
 [25.        4.        4.        3.        4.        3.        3.
   2.        2.        4.        5.        5.        2.        1.
   3.        4.        4.        4.        4.        8.        4.
   2.        1.        2.        9.11528  39.45391 ]
 [23.        4.        4.        4.        4.        4.        2.
   3.        4.        4.        4.        4.        1.        0.
   4.        4.        2.        4.        4.        7.        4.
   3.        2.        2.       10.340347 43.852825]
 [23.        4.        4.        4.        4.        4.        2.
   3.        4.        4.        4.        4.        1.        0.
   4.        4.     

## Deformazione delle variabili
Facciamo valere le preferenze

In [408]:
exps = [(1/5), (1/1.3), 1, 2, 5]
for i in range(1,6):
    X[:,i] = np.int64(np.float_power((11-X[:,i]), exps[int(weights[i-1])]))
X[-1,:] = 0
print(X[:,1:6])

[[4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [4. 4. 4. 4. 1.]
 [0. 0. 0. 0. 0.]]


## Scaling

In [409]:
X = robust_scale(X)
print(X)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -7.05430321e-01  8.63597626e-02]
 [ 2.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00 -1.00000000e+00
  -2.00000000e+00  0.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00 -1.00000000e+00  0.00000000e+00
   2.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00 -1.00000000e+00 -1.00000000e+00  0.00000000e+00
  -4.27893722e-01 -4.17734259e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.0

## Nearest Neighbor
È arrivato il momento tanto atteso di trovare i punti più simili a quello di ricerca

In [410]:
nn = NearestNeighbors(n_jobs=-1)
nn.fit(X[:-1])

dsts, idxs = nn.kneighbors(X[-1].reshape(1,-1), return_distance=True)
print("Indici:",idxs)
print("Distanze:",dsts)

Indici: [[ 1 11 14 10  2]]
Distanze: [[45.71417065 46.36833307 46.96790158 47.32988947 47.36022221]]


## Risultato

In [411]:
with pd.option_context('display.max_columns', None):
    print("ID punto di ricerca:",IDs[-1])
    print("ID dei punti simili:",IDs[idxs[0]])
    print("Punto di ricerca:",df1[df1["ID"] == IDs[-1]])
    for i in range(idxs.shape[1]-1):
        print(f"{i+1}° punto più simile:",df1[df1["ID"] == IDs[idxs[0,i]]])

ID punto di ricerca: 7543
ID dei punti simili: [ 533 5741 7854 5626 2731]
Punto di ricerca:         ID  age_o  attractive_important  sincere_important  \
7543  7543   24.0                   1.0                1.0   

      intellicence_important  funny_important  ambtition_important   age  \
7543                     1.0              1.0                  0.0  29.0   

      attractive  sincere  intelligence  funny  ambition  gender  sports  \
7543         2.0      5.0           4.0    5.0       4.0     1.0     2.0   

      tvsports  exercise  dining  museums  art  hiking  gaming  clubbing  \
7543       2.0       2.0     4.0      5.0  1.0     0.0     0.0       4.0   

      reading   tv  theater  movies  concerts  music  shopping  yoga  arts  \
7543      4.0  4.0      4.0     4.0       8.0    4.0       4.0   2.0   2.0   

      longitude   latitude age_interval gender_text gender_partner_text  
7543   8.734348  45.157944   (18, 32.0)      he,him                 NaN  
1° punto più simile